In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import KFold


In [12]:

classes = 0
def load_n_process():
    df = pd.read_csv("../datasets/census_income/census-income-data.csv", header=None)
    L = preprocessing.LabelEncoder()
    col = 0
    while col < df.shape[1]:
        if ((df[col].dtype == 'int64') or (df[col].dtype == 'float64')):
            dfqn = np.isreal(df.ix[:, col])
            df2 = df[dfqn]
            dfmean = np.mean(df2.ix[:, col])
            df.ix[dfqn != True, col] = dfmean
        else:
            dfqn = df.ix[:, col] != ' ?'
            if dfqn.all() != True:
                df2 = df[dfqn]
                modecol = df2.ix[:, col].mode()[0]
                df.ix[dfqn != True, col] = modecol
            df.ix[:, col] = L.fit_transform(df.ix[:, col])
            df.ix[:,col] = df.ix[:,col].astype('category')
        col += 1

    return df

In [13]:
def calculculateProbs(df):
    probs = {}
    global classes
    classes = np.unique(df.ix[:,41])
    priorProb = {}
    classProb={}

    for cls in classes:
        priorProb[cls] = np.log(df.ix[df.ix[:, 41] == cls, 41].size/float(df.shape[0]))
        col = 0
        colprob={}
        while col < df.shape[1] -1:

            if ((df[col].dtype == 'int64') or (df[col].dtype == 'float64')):
               probd = {}
               dfc = df.ix[df.ix[:, 41] == cls, col]
               probd['mean'] =  dfc.mean()
               probd['variance'] =  dfc.var()
               colprob[col] = probd
            else:

                dfc = df[df.ix[:, 41] == cls]
                cats = np.unique(dfc.ix[:,col])

                catprob = {}
                for cat in cats:
                    catprob[cat] = dfc[dfc.ix[:, col] == cat].size/float(dfc.size)
                colprob[col] = catprob

            col +=1
        classProb[cls] = colprob


    probs['priorProb'] = priorProb
    probs['classCondProb'] = classProb
    return probs


In [14]:
def predict(dataset, probs ):

    global classes
    row =0
    result = []
    while row < dataset.shape[0]:

        posteriorClassProbs = {}
        predictedClass = 0
        for cls in classes:
            posteriorProb = 0
            col=0
            while col < dataset.shape[1]:

                if ((dataset[col].dtype == 'int64') or (dataset[col].dtype == 'float64')):
                    mean = probs['classCondProb'][cls][col]['mean']
                    variance = probs['classCondProb'][cls][col]['variance']
                    x= dataset.iloc[row][col]
                    posteriorProb += (np.square(x-mean)/(float)(2*variance) + np.log(float(np.sqrt(2*np.pi*variance))))*(-1)
                else:
                    cat = dataset.iloc[row][col]
                    if probs['classCondProb'][cls][col].has_key(cat):
                        posteriorProb += np.log(probs['classCondProb'][cls][col][cat])
                col += 1
            posteriorProb += probs['priorProb'][cls]
            posteriorClassProbs[posteriorProb] = cls

        posteriorClassProb = 0
        for prob in posteriorClassProbs.keys():
            if posteriorClassProb == 0 or posteriorClassProb < prob:
                posteriorClassProb =  prob
                predictedClass = posteriorClassProbs[prob]


        result.append(predictedClass)
        row += 1
    return result


df = load_n_process()


In [17]:
kf = KFold(n_splits=10)
accuracy = []
for train, test in kf.split(df):
    X_train, X_test = df.ix[train,:], df.ix[test, :]
    Y_test = X_test.ix[:,41]
    Y_test = Y_test.astype('int')
    Y_test = pd.DataFrame(Y_test)
    probs = calculculateProbs(X_train)
    accuracy.append(100 - (np.sum(np.abs(Y_test[41] - predict(X_test.ix[:,:-1], probs)))/float(Y_test.shape[0]))*100)

avg_accuracy = np.mean(accuracy)
std_deviation = np.std(accuracy)
print avg_accuracy
print std_deviation

93.7941978907
0.138985693473


In [19]:
for i in range(1,3):               
    kf = KFold(n_splits=10)
    accuracy=[]
    accuracy_iter = []
    for train, test in kf.split(df):
        X_train, X_test = df.ix[train,:], df.ix[test, :]
        Y_test = X_test.ix[:,41]
        Y_test = Y_test.astype('int')
        Y_test = pd.DataFrame(Y_test)
        probs = calculculateProbs(X_train)
        accuracy_iter.append(100 - (np.sum(np.abs(Y_test[41] - predict(X_test.ix[:,:-1], probs)))/float(Y_test.shape[0]))*100)
    accuracy_iter_mean = np.mean(accuracy_iter)
    std_deviation = np.sqrt(np.var(accuracy_iter))
    accuracy.append(accuracy_iter_mean)
acc_mean = np.mean(accuracy)
acc_std = np.std(accuracy)

print("acc : {}".format(acc_mean))
print("std : {}".format(acc_std))   



acc : 93.7941978907
std : 0.0
